# Droplets

This *jupyter* notebook carries out the analyses in the upcoming journal article focusing on analyzing gravitationally unbound, coherent structures with significant velocity gradients.  The notebook is edited to work with data and *Python* scripts in the [Github repo](https://github.com/hopehhchen/Droplets).

## 0. Abstract and Outlines

The project looks for coherent cores with significant velocity gradients in the L1688 region in Ophiuchus and the B18 region in Taurus, out of the four nearby star forming regions covered by the *GBT Ammonia Survey* (GAS) Data Release 1 ([Friesen and Pineda et al., 2017](https://ui.adsabs.harvard.edu/#abs/2017ApJ...843...63F/abstract)).  One goal of the project is to update numbers for physical properties of (potentially) rotational motions within these structures.  The relation between the velocity gradient and the size was first examined using observations of ammonia molecular line emission by [Goodman et al. (1993)](https://ui.adsabs.harvard.edu/#abs/1993ApJ...406..528G/abstract).  With an improved physical resolution of ~4000 AU (the FWHM beam size at the distance of Ophiuchus and Taurus), we hope to provide more reliable and relevant numbers for simulations and analytical models, especially ones concerning with disk formation inside star forming cores.

Subsequent analyses included in this project discover that the structures are possibly gravitationally unbound, despite subsonic velocity dispersions and relatively high column densities.  The behavior is unexpected in the final stages of monolithic star formation driven by gravity.  Due to their small sizes and the seeming unboundness, we term these structures "droplets," to distinguish them from "cores," which are often used to indicate bound and star forming, sub-parsec structures.  The project aims at providing a guess to how the "droplets" are formed and what role they could play in the star formation process.

This notebook is organized as follows. In ***I. Identification of Structures***, we go through how we identify these structures and how we define the boundaries.  In ***II. Physical Properties of Droplets***, we derive the basic physical properties of the "droplets," based on the property maps from ammonia line fitting to the GAS DR1 data.  In ***III. Kinematics and Dynamics***, we look for possible explanations to the formation of "droplets" and the role they play in star formation.  In particular, we look into the spatial distribution of ram pressure within and around the "droplets," to see whether the droplets could be pressure-bound.  In all sections of this notebook, we also provide the codes to generate tables and plots that eventually appear in the corresponding parts of the article.

Further discussions are included in the article hosted on Authorea.  **This notebook, though filled with definitions essential for users to understand the analyses, is *not* meant to be complete or continuous as a journal article itself.**  Please look in the article for details.

***CAUTION: This is work in progress.***

[*The square brackets in this notebook host the technical explanations of following code cells.  They should be treated as code comments.*]

#### Authors
* **Hope Chen** (leading and corresponding, at <hhchen@cfa.harvard.edu>; Harvard-Smithsonian Center for Astrophysics)
* Jaime Pineda (Max-Planck-Institut für Extraterrestrische Physik)
* Alyssa Goodman (Harvard-Smithsonian Center for Astrophysics)
* Andreas Burkert (University Observatory Munich)

#### Code Dependencies
* `numpy`
* `scipy`
* `astropy`
* `FITS_tools`
* `matplotlib`
* `pandas`

In [1]:
%matplotlib inline
import sys
import os

#
import numpy as np

#
from astropy.io import fits
import astropy.wcs as wcs
import astropy.units as u
import astropy.constants as c
import FITS_tools as fits_tools

#
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#
import pandas as pd

## 
import styles

## 0.1 Data and Targets

The GAS DR1 inlcudes observations of the ammonia hyperfine line emission made with the *Green Bank Telescope* (GBT).  By fitting Guassian profiles to the ammonia hyperfine lines, [Friesen and Pineda et al. (2017)](https://ui.adsabs.harvard.edu/#abs/2017ApJ...843...63F/abstract) derive the property maps of four nearby star forming regions: B18 in Taurus, L1688 in Ophiuchus, NGC1333 in Perseus, and Orion A North.  These physical properties include the NH$_3$ velocity centroid (in the LSR frame; $V_\text{LSR}$), the NH$_3$ velocity dispersion ($\sigma_V$), the NH$_3$ column density (N$_{\text{NH}_3}$), the excitation temperature of NH$_3$ hyperfine lines (T$_\text{ex}$), and the kinetic temperature (T$_\text{kin}$).  We look for coherent structures using these physical properties in the B18 region and the L1688 region, where their proximity (at the distances of ~135 pc) grants us the highest physical resolution (the GBT FWHM beamsize of 32" at 23 GHz corresponds to ~4300 AU).

The GAS data are supplemented by the column density and dust temprature maps derived using *Herschel* observations of dust continuum emission.  The *Herschel* observations were made as part of the *Herschel Gould Belt Survey* (GBS; [André et al., 2010](https://ui.adsabs.harvard.edu/#abs/2010A&A...518L.102A/abstract)), and the data are from the *Herschel* Science Archive.  The FWHM beamsize of SPIRE 500 µm is 36", which matches well with the GBT beamsize.

The full GAS DR1 dataset is hosted on [Dataverse](https://dataverse.harvard.edu/dataverse/GAS_Project).  In the [Droplets gihub](https://github.com/hopehhchen/Droplets), only data that are needed in the following analyses are included.

[*The datasets are read and stored in `dict_data`.*]

In [26]:
# data folder (within the current directory)
direcData = os.getcwd()+'/data/'
## lists of property maps to be read in
list_propertiesGAS = ['Vlsr', 'Sigma', 'N_NH3', 'Tex', 'Tkin']
list_propertiesHerschel = ['colden', 'temp']


# Read the data.
dict_data = {'L1688': {}, 'B18': {}}
for reg in ['L1688', 'B18']:
    
    # GAS DR1
    for prop in list_propertiesGAS:
        dict_data[reg][prop] = fits.open(direcData+'GAS_DR1/'+reg+'/'+reg+'_'+prop+'_DR1_rebase3_flag.fits')[0].data
    ## header
    dict_data[reg]['header_GAS'] = fits.open(direcData+'GAS_DR1/'+reg+'/'+reg+'_Vlsr_DR1_rebase3_flag.fits')[0].header
        
    # Herschel
    for prop in list_propertiesHerschel:
        dict_data[reg][prop] = fits.open(direcData+'Herschel/'+reg+'/'+reg+'_'+prop+'_masked.fits')[0].data
    ## header
    dict_data[reg]['header_Herschel'] = fits.open(direcData+'Herschel/'+reg+'/'+reg+'_colden_masked.fits')[0].header


For easier comparison between the GAS DR1 data and the *Herschel* maps, we project the *Herschel* maps to the GAS DR1 grid.  As mentioned above, the GBT has a FWHM beamsize at 23 GHz (32") similar to the FWHM beamsize of *Herschel* 500 µm (36").  Thus, we can safely assume that the regridding does not introduce a significant biase.

[*The regridding is done using the *Python* package [`FITS_tools`](http://fits-tools.readthedocs.io/en/latest/fits_tools.html).  The newly projected images replace the original images in `dict_data`.*]

In [36]:
# Regrid the Herschel maps.
for reg in ['L1688', 'B18']:
    
    for prop in list_propertiesHerschel:
        
        # Regrid using hcongrid.
        image = dict_data[reg][prop].copy()
        header1 = dict_data[reg]['header_Herschel'] ## original projection
        header2 = dict_data[reg]['header_GAS'] ## projection to interpolate into
        
        dict_data[reg][prop] = fits_tools.hcongrid.hcongrid(image, header1, header2)

## I. Identification of Structures

In [42]:
!ls /Users/hopechen/Documents/projects/Oph/data/FCRAO/

OphA_12coFCRAO_F_map.fits         OphA_13coFCRAO_F_sigGaussian.fits
OphA_12coFCRAO_F_xyv.fits         OphA_13coFCRAO_F_xyv.fits
OphA_12coFCRAO_F_xyv_base2.fits   OphA_13coFCRAO_F_xyv_base2.fits
OphA_12coFCRAO_F_xyv_fixed.fits   OphA_13coFCRAO_F_xyv_fixed.fits
OphA_13coFCRAO_F_Moment2.fits     OphA_13coFCRAO_F_xyv_regrid.fits
OphA_13coFCRAO_F_map.fits


In [2]:
direc = '/Users/hopechen/Documents/projects/Oph/data/FCRAO/'
test = fits.open(direc+'OphA_13coFCRAO_F_xyv_base2.fits')[0]

In [3]:
vaxis = test.header['CRVAL3']+np.arange(test.header['NAXIS3'])*test.header['CDELT3']

In [4]:
vcube = np.broadcast_to(vaxis, (1087, 750, 527))